- data source: http://www.manythings.org/anki/

- This file also works. Still no pre-trained embeddings. 
- Used functions for code blocks. But that didn't really make things much neater. 
- Bidirectional LSTM for encoder
- In part 6, incorporate pre-trained embeddings

In [2]:
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, RNN, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda, Embedding, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.initializers import Constant
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np
import pandas as pd
import string 
import regex as re

from sklearn.model_selection import train_test_split

import io
import spacy

import warnings 
warnings.filterwarnings('ignore')

# from tensorflow.python.framework.ops import disable_eager_execution
# disable_eager_execution()

/opt/anaconda3/envs/deep_learning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [68]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# df_en_de = pd.read_table('/content/gdrive/MyDrive/deu-eng/deu.txt', names=['eng', 'deu', 'attr'])

In [3]:
df_en_de = pd.read_table('deu-eng/deu.txt', names=['eng', 'deu', 'attr'])

In [4]:
df_en_de = df_en_de.drop('attr',axis = 1).rename(columns = {'eng':'english', 'deu':'german'})

In [5]:
# Lowercase all characters
df_en_de['english'] = df_en_de['english'].apply(lambda x: x.lower())
df_en_de['german'] = df_en_de['german'].apply(lambda x: x.lower())

# Remove quotes
df_en_de['english'] = df_en_de['english'].apply(lambda x: re.sub("'", '', x))
df_en_de['german'] = df_en_de['german'].apply(lambda x: re.sub("'", '', x))

# Set of all special characters
exclude = set(string.punctuation) 

# Remove all the special characters
df_en_de['english'] = df_en_de['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
df_en_de['german']=df_en_de['german'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Add start and end tokens to target sequences
df_en_de['german'] = df_en_de['german'].apply(lambda x : 'START_ '+ x + ' _END')

In [6]:
max_len = 10

pairs = df_en_de
pairs['english_length'] = pairs['english'].apply(lambda x: len(x.split(' ')))
pairs['german_length'] = pairs['german'].apply(lambda x: len(x.split(' ')))

pairs = pairs[pairs['english_length'] <= max_len]
pairs = pairs[pairs['german_length'] <= max_len]
print(len(pairs))
pairs = pairs.sample(frac = 0.1)
print(len(pairs))

209317
20932


In [7]:
# Vocabulary of English
all_en_words=set()
for eng in pairs['english']:
    for word in eng.split():
        if word not in all_en_words:
            all_en_words.add(word)

# Vocabulary of German 
all_de_words=set()
for de in pairs['german']:
    for word in de.split():
        if word not in all_de_words:
            all_de_words.add(word)

# Max Length of source sequence
length_list=[]
for l in pairs['english']:
    length_list.append(len(l.split(' ')))
max_length_src = np.max(length_list)

# Max Length of target sequence
length_list=[]
for l in pairs['german']:
    length_list.append(len(l.split(' ')))
max_length_tar = np.max(length_list)


input_words = sorted(list(all_en_words))
target_words = sorted(list(all_de_words))

# Calculate Vocab size for both source and target
# Add 1 for zero padding
num_encoder_tokens = len(all_en_words) + 1
num_decoder_tokens = len(all_de_words) + 1

# Create word to token dictionary for both source and target
#input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
#target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
input_word_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_word_index = dict([(word, i+1) for i, word in enumerate(target_words)])

# Create token to word dictionary for both source and target
# reverse_input_token_index = dict((i, word) for word, i in input_token_index.items())
# reverse_target_token_index = dict((i, word) for word, i in target_token_index.items())
input_index_word = dict((i, word) for word, i in input_word_index.items())
target_index_word = dict((i, word) for word, i in target_word_index.items())

In [8]:
X, y = pairs['english'], pairs['german']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=101)

In [9]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):          # j = batch number
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32') # (m, max_len)
            
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32') # (m, max_len)

            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')    # (m, max_len, num_decoder_tokens)
            
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_word_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_word_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_word_index[word]] = 1. 
                        """ This should be target_token_index[word] - 1"""
            # decoder_target_data = np.transpose(decoder_target_data, axes = [1, 0, 2])
            # decoder_target_data = list(decoder_target_data)
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [10]:
num_decoder_tokens

10258

- global variables for one_step_attention:

In [68]:
Tx = max_len
Ty = max_len
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis = -1)
densor1 = Dense(10, activation = 'tanh')
densor2 = Dense (1, activation = 'relu')
dotor = Dot(axes = 1)

In [69]:
def one_step_attention (h, s_prev):

    # calculate the Context vector for one time-step of decoder

    # h = (m, Tx, n_h)
    # s_prev = (m, n_s)
    # returns: context – we will then use [context; y_prev] as input of Decoder

    s_prev = repeator(s_prev)                   # (m, Tx, n_s)
    concat = concatenator([h, s_prev])          # (m, Tx, n_h + n_s)
    e = densor1 (concat)                        # (m, Tx, 10)
    energies = densor2 (e)                      # (m, Tx, 1)
    alphas = tf.nn.softmax(energies, axis = 1)  # (m, Tx, 1)
    context = dotor([alphas, h])                # alphas = (m, Tx, 1)
                                                # h = (m, Tx, n_h)
                                                # (m, 1, n_h)
    return context                              # (m, 1, n_h)

In [70]:
x_emb_dim = 300
y_emb_dim = 300

n_h = 200
n_s = 200


- global variables for pre-attention LSTM:

In [71]:
x_inputs = Input(shape = (Tx,))                         # (None, Tx) = (m, Tx)
x_emb_layer = Embedding(num_encoder_tokens, x_emb_dim, mask_zero = True)  
x_emb = x_emb_layer(x_inputs)                         # (None, Tx, x_emb_dim) = (m, Tx, x_emb_dim)

# Use bidirectional LSTM
enc_lstm_layer = Bidirectional(LSTM(n_h, dropout = 0.3, recurrent_dropout = 0.3, return_sequences=True, return_state = True))
h_enc, s_enc, c_enc, _, _ = enc_lstm_layer(x_emb)                             # h_enc = (None, Tx, n_h) = (m, Tx, 2*n_h)
                                                                


- global variables for post-attention LSTM:

In [72]:
y_inputs = Input(shape = (Ty,))                         # (None, Ty) = (m, Ty)
y_emb_layer = Embedding (num_decoder_tokens, y_emb_dim, mask_zero = True)
y_emb = y_emb_layer(y_inputs)                         # (None, Ty, y_emb_dim) = (m, Ty, y_emb_dim)


dec_lstm_layer = LSTM(n_s, dropout = 0.3, recurrent_dropout = 0.3, return_state = True)
output_layer = Dense(num_decoder_tokens, activation='softmax')



In [73]:
print(y_inputs.shape[0])

None


In [74]:
def training_model():

    # s<0> and c<0> for decoder = s<Ty> and c<Ty> for encoder
                                                                 
    s_dec = s_enc
    c_dec = c_enc
    
    outputs = []
    for t in range(Ty):
        context = one_step_attention(h_enc, s_dec)                                            # context = (m, 1, n_h)
        concat = Concatenate(axis = -1)([context, tf.expand_dims(y_emb[:,t,:],1)])            # concat = (m, 1, n_h + y_emb_dim)
        
        # update decoder LSTM hidden state (s) and cell state (c)
        _, s_dec, c_dec = dec_lstm_layer (initial_state = [s_dec, c_dec], inputs = concat)        # s = (None, Ty, n_s)
        
        # pass decoder LSTM hidden state (s) through output layer to get y prediction
        out = output_layer(s_dec)                                                                   # out = (m, num_decoder_tokens)
        outputs.append(out)
                                                                
    outputs = tf.stack(outputs, axis = 1)
    training_model = Model(inputs = [x_inputs, y_inputs], outputs = outputs)

    return training_model

In [75]:
training_model = training_model()
training_model.compile(optimizer= 'Adam', loss='categorical_crossentropy', metrics=['acc'])

train_samples = len(X_train) # Total Training samples
val_samples = len(X_test) # total validation samples
batch_size = 128
epochs = 5


In [76]:
history = training_model.fit(generate_batch(), 
                    steps_per_epoch=train_samples//batch_size, 
                    epochs = 5,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples // batch_size, 
                    verbose = 1)

Epoch 1/5
130/130 [==============================] - 68s 350ms/step - loss: 4.1713 - acc: 0.1149 - val_loss: 3.7617 - val_acc: 0.1229
Epoch 2/5
130/130 [==============================] - 42s 324ms/step - loss: 3.6530 - acc: 0.1244 - val_loss: 3.6291 - val_acc: 0.1316
Epoch 3/5
130/130 [==============================] - 43s 327ms/step - loss: 3.4597 - acc: 0.1416 - val_loss: 3.4686 - val_acc: 0.1542
Epoch 4/5
130/130 [==============================] - 42s 325ms/step - loss: 3.2604 - acc: 0.1653 - val_loss: 3.2953 - val_acc: 0.1738
Epoch 5/5
130/130 [==============================] - 44s 336ms/step - loss: 3.0535 - acc: 0.1849 - val_loss: 3.1358 - val_acc: 0.1933


In [82]:
history2 = training_model.fit(generate_batch(), 
                    steps_per_epoch=train_samples//batch_size, 
                    epochs = 10,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples // batch_size, 
                    verbose = 1)

Epoch 1/10
130/130 [==============================] - 41s 312ms/step - loss: 2.8784 - acc: 0.2040 - val_loss: 3.0066 - val_acc: 0.2073
Epoch 2/10
130/130 [==============================] - 41s 313ms/step - loss: 2.7094 - acc: 0.2190 - val_loss: 2.8964 - val_acc: 0.2186
Epoch 3/10
130/130 [==============================] - 41s 315ms/step - loss: 2.5601 - acc: 0.2331 - val_loss: 2.8040 - val_acc: 0.2301
Epoch 4/10
130/130 [==============================] - 41s 313ms/step - loss: 2.4246 - acc: 0.2469 - val_loss: 2.7179 - val_acc: 0.2409
Epoch 5/10
130/130 [==============================] - 41s 314ms/step - loss: 2.2946 - acc: 0.2609 - val_loss: 2.6456 - val_acc: 0.2515
Epoch 6/10
130/130 [==============================] - 43s 329ms/step - loss: 2.1744 - acc: 0.2742 - val_loss: 2.5834 - val_acc: 0.2593
Epoch 7/10
130/130 [==============================] - 47s 364ms/step - loss: 2.0574 - acc: 0.2877 - val_loss: 2.5259 - val_acc: 0.2675
Epoch 8/10
130/130 [==============================] - 4

In [88]:
history3 = training_model.fit(generate_batch(), 
                    steps_per_epoch=train_samples//batch_size, 
                    epochs = 10,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples // batch_size, 
                    verbose = 1)

Epoch 1/10
130/130 [==============================] - 42s 319ms/step - loss: 1.6588 - acc: 0.3332 - val_loss: 2.3520 - val_acc: 0.2935
Epoch 2/10
130/130 [==============================] - 41s 315ms/step - loss: 1.5674 - acc: 0.3434 - val_loss: 2.3286 - val_acc: 0.2965
Epoch 3/10
130/130 [==============================] - 41s 317ms/step - loss: 1.4857 - acc: 0.3539 - val_loss: 2.2861 - val_acc: 0.3035
Epoch 4/10
130/130 [==============================] - 41s 317ms/step - loss: 1.4070 - acc: 0.3646 - val_loss: 2.2657 - val_acc: 0.3076
Epoch 5/10
130/130 [==============================] - 41s 316ms/step - loss: 1.3320 - acc: 0.3754 - val_loss: 2.2454 - val_acc: 0.3115
Epoch 6/10
130/130 [==============================] - 41s 319ms/step - loss: 1.2578 - acc: 0.3871 - val_loss: 2.2239 - val_acc: 0.3150
Epoch 7/10
130/130 [==============================] - 41s 313ms/step - loss: 1.1868 - acc: 0.3988 - val_loss: 2.2097 - val_acc: 0.3177
Epoch 8/10
130/130 [==============================] - 4

In [89]:
def encoder_model():
    # x_inputs defined above as Input
    # h_enc, s_enc, c_enc also defined above
    encoder_model = Model(inputs = x_inputs, outputs = [h_enc, s_enc, c_enc])
    return encoder_model

def decoder_model():
    # The below three decoder inputs will come from encoder_model.predict()
    decoder_input_h = Input(shape = (Tx, 2*n_h))               # (None, Tx, 2*n_h) 
    decoder_input_s = Input(shape=(n_h, ))                    # (None, n_h) 
    decoder_input_c = Input(shape = (n_h,))                   # (None, n_h)

    # y_emb_2 will be our y_pred at t-1
    y_inp_2 = Input(shape = (None,))                        # (None, None) = (m, Ty)
    y_emb_2 = y_emb_layer(y_inp_2)                          # (None, None, y_emb_dim) = (m, Ty, y_emb_dim)

    # Use decoder_input_s and decoder_input_h to compute context vector
    context = one_step_attention(decoder_input_h, decoder_input_s)    # (m, 1, 2*n_h)

    # concatenate context with y_emb_2
    concat2 = Concatenate(axis = -1)([context, tf.expand_dims(y_emb_2[:,-1,:],1)])                       
                                                            # concat2 = (None, 1, 2*n_h + y_emb_dim)

    # Feed concat2 as input; decoder_input_s and decoder_input_c as initial state
    _, decoder_output_s, decoder_output_c = dec_lstm_layer (
                                                            initial_state = [decoder_input_s, decoder_input_c], 
                                                            inputs = concat2
                                                            )     
                                                # decoder_output_s = (None, n_s) 
                                                # decoder_output_c = (None, n_s) 


    decoder_output_y = output_layer(tf.expand_dims(decoder_output_s,1))           # (None, 1, num_decoder_tokens)

    decoder_model = Model(inputs = [decoder_input_h, decoder_input_s, decoder_input_c, y_inp_2],
                            outputs = [decoder_output_y, decoder_output_s, decoder_output_c])
    return decoder_model

In [90]:
encoder_model = encoder_model()
decoder_model = decoder_model()

In [91]:
def decode_sequence(input_sequence):

    # input_sequence = (1, max_len)
    
    # get hidden states + final hidden state + final cell state from encoder 
    h_enc_pred, s_enc_pred, c_enc_pred = encoder_model.predict(input_sequence)
    
    # define y_pred at time 0    
    target_seq = np.zeros((1,1))
    target_seq[0,0] = target_word_index['START_']

    stop_condition = False
    decoded_sentence = ''

    # initialise hidden state and cell state input for decoder
    decoder_s_pred = s_enc_pred                         # (None, n_h) = (m, n_h)
    decoder_c_pred = c_enc_pred                         # (None, n_h) = (m, n_h)
    
    while not stop_condition:
        decoder_y_pred, decoder_s_pred, decoder_c_pred = decoder_model.predict([h_enc_pred, decoder_s_pred, decoder_c_pred, target_seq])  
        y_index = np.argmax(decoder_y_pred[0,-1,:])
        y_word = target_index_word[y_index]
        decoded_sentence += ' ' + y_word

        # Exit condition: either hit max length
        # or find stop character.
        if (y_word == '_END' or
           len(decoded_sentence.split()) > max_len):
            stop_condition = True
        
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = y_index
        
        
        
    return decoded_sentence


In [92]:
def sentence_to_seq(sentence):
    """
    sentence = string
    """
    
    encoder_input_data = np.zeros((1, max_len))     # (1, max_len)
    
    sentence = sentence.lower().split()
    #print(sentence)
    for j, word in enumerate(sentence):
        encoder_input_data[0,j] = input_word_index[word]        # (1, max_len)
    # print(encoder_input_data.shape)
    return encoder_input_data                           # (1, max_len)


In [81]:
sentences = list(X_train.iloc[:10].values)          # list of sentences

translations = []
for sentence in sentences:
    seq = sentence_to_seq(sentence)             # seq = (1, max_len)
    #print(seq, seq.shape)
    translation = decode_sequence(seq)
    translations.append(translation)

sentence_translation_pairs = zip (sentences, translations)
for elem in sentence_translation_pairs:
    print (elem)

0
0
0
0
0
0
0
0
0
0
('the asian part of russia is larger than china', ' die vater ist die haus ist _END')
('were conservative', ' sie ist _END')
('we live in the age of technology', ' er ist die paar haus _END')
('my brother is stupid', ' sie ist ein sehr _END')
('tom spent much of his time reading', ' tom hat die paar haus _END')
('mary doesnt like men who look like tom', ' er hat tom nicht tom nicht _END')
('tom quickly changed the subject', ' tom hat die sehr sehr _END')
('she is certain to come on time', ' sie ist die arbeit zu _END')
('please listen to us', ' wir ist das _END')
('this amount includes tax', ' sie ist ein sehr _END')


In [87]:
sentences = list(X_train.iloc[:10].values)          # list of sentences

translations = []
for sentence in sentences:
    seq = sentence_to_seq(sentence)             # seq = (1, max_len)
    #print(seq, seq.shape)
    translation = decode_sequence(seq)
    translations.append(translation)

sentence_translation_pairs = zip (sentences, translations)
for elem in sentence_translation_pairs:
    print (elem)

('the asian part of russia is larger than china', ' der katze ist in dem zimmer ist als _END')
('were conservative', ' wir sind _END')
('we live in the age of technology', ' wir haben die straße des wand _END')
('my brother is stupid', ' mein vater ist sehr _END')
('tom spent much of his time reading', ' tom hat mit einer woche mit tom _END')
('mary doesnt like men who look like tom', ' tom hat maria nicht wie maria haben _END')
('tom quickly changed the subject', ' tom hat sich die ganzen leben zu sein _END')
('she is certain to come on time', ' sie hat sich nicht zu zu gehen _END')
('please listen to us', ' bitte bitte uns zu _END')
('this amount includes tax', ' das haus ist in diesem zimmer _END')


In [93]:
sentences = list(X_train.iloc[:10].values)          # list of sentences

translations = []
for sentence in sentences:
    seq = sentence_to_seq(sentence)             # seq = (1, max_len)
    #print(seq, seq.shape)
    translation = decode_sequence(seq)
    translations.append(translation)

sentence_translation_pairs = zip (sentences, translations)
for elem in sentence_translation_pairs:
    print (elem)

('the asian part of russia is larger than china', ' der dieb des landes des feuers ist größer _END')
('were conservative', ' wir sind spontan _END')
('we live in the age of technology', ' wir leben im zeitalter der treppe _END')
('my brother is stupid', ' mein bruder ist wütend _END')
('tom spent much of his time reading', ' tom hat viel zeit mit lange zeit _END')
('mary doesnt like men who look like tom', ' tom liebt die plan ist uns wie tom _END')
('tom quickly changed the subject', ' tom hat seine gefühle nicht die geduld _END')
('she is certain to come on time', ' sie ist offensichtlich dass sie da kommt _END')
('please listen to us', ' bitte hör uns zu _END')
('this amount includes tax', ' in diesem betrag ist die grenze _END')
